**Plase Open In nbview using the following link**
https://nbviewer.jupyter.org/github/wuhao21/15688_tutorial/blob/master/DotA2_tutorial.ipynb

## Introduction

This tutorial will introduce the [Valve's Dota 2 API](https://wiki.teamfortress.com/wiki/WebAPI#Dota_2) and several methods to analyze a Dota 2 match and a Dota 2 player.

[Dota 2](http://www.dota2.com/play/) is a multiplayer online battle arena ([MOBA](https://en.wikipedia.org/wiki/Multiplayer_online_battle_arena)) video game developed and published by Valve Corporation. Dota 2 is played in matches between two teams of five players, with each team occupying and defending their own separate base on the map. Each of the ten players independently controls a powerful character, known as a "hero", who all have unique abilities and different styles of play.

According to Valve, there are more than 1,000,000 active Dota 2 players in the world and a huge amount of matches being played every day. Many Dota 2 professional teams are competing with each other in for huge bonus and honor. For instance, the sixth [Dota 2 TI](https://en.wikipedia.org/wiki/The_International_%28Dota_2%29) had an overall prize pool of over $20 million. By analyzing the Dota 2 match data, we could evaluate the performance of each player and find the most valuable player on the winning team and drawbacks of losing teams. In this way, players and professional teams are able to improve their skills efficiently. Moreover, by analyzing the hero preference of players, we could design specific team formation against our opponents.

### Tutorial content
In this tutorial, we will first introduce [Dota 2 APIs](#Work-with-Dota-2-APIs) and show how to collect data of:
- [Hero information](#Collecting-hero-information)
- [a Dota2 match](#Collecting-data-of-a-Dota2-match)
- [Match history of a player](#Collecting-match-history-of-a-player)

Also, we will cover following analyzing method:
- [Analyze a match: Who is MVP?](#Analyze-a-match:-Who-is-MVP?)
- [Analyze a player: Favorite Heroes](#Analyze-a-player:-Favorite-Heroes)

## Work with Dota 2 APIs

### Installing the libraries

We are going to use the similar libraries as when we are dealing with Yelp APIs in assignment 1. The only new library you need to install is [tqdm](https://pypi.python.org/pypi/tqdm). It is an easy-to-use progress meter for the python script. Since the response time of Dota 2 APIs is relatively long, tqdm could let us know the exact progress and estimated running time. You can install it using `pip`:

    $ pip install --upgrade tqdm
    
After you finished the installation, please make sure the following commands are workable:

In [2]:
import json
import requests
import numpy as np
import time, datetime
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt

### Getting an API Key

First of all, you need to apply for an API key from [Valve](https://steamcommunity.com/dev/apikey). Please register a steam account if you don't have one. 

We would store the key in a file called `api_key.txt` (run in terminal):
```bash
echo 'DOTA2_API_KEY' > api_key.txt
```

Then, by <cite>the following function (cited from 15688 assignment 1)</cite>, we could read in the key to authenticate with the API by adding it to the request parameters.

In [3]:
def read_api_key(filepath):
    with open('api_key.txt', 'r') as f:
        return f.read().replace('\n','')

In [4]:
api_key = read_api_key('api_key.txt')

## Collecting hero information

Using [`WebAPI/GetHeroes`](https://wiki.teamfortress.com/wiki/WebAPI/GetHeroes), we could get the list of heroes in JSON format:
```python
{
    'result' :
    {
        'heroes' : #List of heroes
        'name' : #The tokenized string for the name of the hero
        'id' : #ID of the hero
        'localized_name' : #The localized name of the hero for use in name display
        'count' : #Number of results
    }
}
```
In the following function, we will:
1. download the JSON data.
2. generate a mapping from hero id to list index since they are not the same. ( ```hereos[8]['hero_id']``` may not equal to 8 )
3. add the ```portrait_url``` field which contains the URL to the png of hero portraits. We can use these portraits in later visualizations.

Originally, the hero name is like `npc_dota_hero_[name]`. For example, the given name of hero Anti-Mage is `npc_dota_hero_antimage`. Correspondingly, the URL of its small portrait is http://cdn.dota2.com/apps/dota2/images/heroes/antimage_sb.png. Therefore, we need to cut `npc_dota_hero_` from the given name in the JSON file and build up the URL.

`get_heroes` will return the processed hero list and hero-id-to-index mapping dictionary at the same time.

In [5]:
def get_heroes(api_key):
    portrait_url_base = 'http://cdn.dota2.com/apps/dota2/images/heroes/'
    url = 'http://api.steampowered.com/IEconDOTA2_570/GetHeroes/v1'
    params = {"key" : api_key, 'language' : 'en'}
    response = requests.get(url, params=params)
    data = json.loads(response.text)['result']
    idx = 0
    heroid2idx = {}
    for hero in data['heroes']:
        heroid2idx[hero['id']] = idx
        hero['portrait_url'] = portrait_url_base + hero['name'].replace('npc_dota_hero_', '') + '_sb.png'
        idx += 1
    return (data['heroes'], heroid2idx)

In [6]:
heroes, heroid2idx = get_heroes(api_key)

## Collecting data of a Dota2 match

Using [`WebAPI/GetMatchDetails`](https://wiki.teamfortress.com/wiki/WebAPI/GetMatchDetails), with a match id, we could get the detailed information of this Dota 2 match:
```python
{
'result' :
    {
    'players' : #List of players in the match.
        [{
            'account_id' : #32-bit account ID
            'hero_id' : #The hero's unique ID.
            'kills' : #The amount of kills attributed to this player.
            'deaths' : #The amount of times this player died during the match.
            'assists' : #The amount of assists attributed to this player.
            'gold' : #The amount of player's remaining gold at the end of the match.
            'last_hits' : #The amount of last-hits the player got during the match.
            'denies' : #The amount of denies the player got during the match.
            'gold_per_min' : #The player's overall gold/minute.
            'xp_per_min' : #The player's overall experience/minute.
            'gold_spent' : #The amount of gold the player spent during the match.
            'hero_damage' : #The amount of damage the player dealt to heroes.
            'tower_damage' : #The amount of damage the player dealt to towers.
            'hero_healing' : #The amount of health the player had healed on heroes.
            'level' : #The player's level at match end.
            ...
        }],
    'duration' : #The length of the match, in seconds since the match began.
    'start_time' : #Unix timestamp of when the match began.
    'match_id' : #The matches unique ID.
    ...
    }
...
}
```

I only listed the fields that we will use in later parts. Function `get_match_details` will return the `result` part of the JSON response for a given `match_id`.

In [8]:
def get_match_details(match_id, api_key):
    url = 'http://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/v001/'
    params = {"key" : api_key, "match_id" : match_id}
    response = requests.get(url, params=params)
    data = json.loads(response.text)['result']
    return data

In [9]:
match_id = '3777928249'
print(get_match_details(match_id, api_key)['players'][0])
print(get_match_details(match_id, api_key)['start_time'])

{'account_id': 148440518, 'player_slot': 0, 'hero_id': 89, 'item_0': 0, 'item_1': 181, 'item_2': 214, 'item_3': 36, 'item_4': 267, 'item_5': 223, 'backpack_0': 188, 'backpack_1': 0, 'backpack_2': 0, 'kills': 4, 'deaths': 9, 'assists': 14, 'leaver_status': 0, 'last_hits': 143, 'denies': 1, 'gold_per_min': 383, 'xp_per_min': 553, 'level': 24, 'hero_damage': 10203, 'tower_damage': 228, 'hero_healing': 0, 'gold': 4196, 'gold_spent': 9155, 'scaled_hero_damage': 5961, 'scaled_tower_damage': 125, 'scaled_hero_healing': 0, 'ability_upgrades': [{'ability': 5469, 'time': 440, 'level': 1}, {'ability': 5468, 'time': 573, 'level': 2}, {'ability': 5469, 'time': 719, 'level': 3}, {'ability': 5467, 'time': 924, 'level': 4}, {'ability': 5469, 'time': 1060, 'level': 5}, {'ability': 5470, 'time': 1093, 'level': 6}, {'ability': 5469, 'time': 1230, 'level': 7}, {'ability': 5467, 'time': 1411, 'level': 8}, {'ability': 5467, 'time': 1482, 'level': 9}, {'ability': 5919, 'time': 1582, 'level': 10}, {'ability':

## Collecting match history of a player

Using [`WebAPI/GetMatchHistory`](https://wiki.teamfortress.com/wiki/WebAPI/GetMatchHistory), we could get match history of a player. Although there are many other options like game mode, skill level, we will only consider the general history without any parameters. The structure looks like:
```python
{
'result' : 
    {
        'matches' : #A list of matches
        [{
            'match_id' : #The matches unique ID
            'players' : #The list of players within the match
            [{
                'account_id' : #32-bit account ID
                'hero_id' : #The hero's unique ID
                ...
            }]
            ...
        }]
        ...
    }
}
```

The maximum amount of matches we could get from this API is 100 every time, at most 500 matches. Therefore, we will need another function to call this API repeatedly to get all history.

In [10]:
def get_match_history(account_id, api_key, game_mode = 1, matches_requested = 25, start_at_match_id = None):
    url = 'http://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/v001/'
    params = {"key" : api_key, \
              "account_id" : account_id, \
              "game_mode" : game_mode, \
              "matches_requested" : matches_requested}
    if (start_at_match_id != None):
        params['start_at_match_id'] = start_at_match_id
    response = requests.get(url, params=params)
    data = json.loads(response.text)['result']
    return data
def get_all_match_history(account_id, api_key, game_mode = 1):
    matches = []
    last_match = None
    results_remaining = 100
    while results_remaining != 0:
        time.sleep(1)
        if (last_match == None):
            data = get_match_history(account_id, api_key, game_mode, matches_requested = 100)
        else:
            data = get_match_history(account_id, api_key, game_mode, \
                                     matches_requested = 100, \
                                     start_at_match_id = last_match)
        results_remaining = data['results_remaining']
        if (matches != []):
            matches = matches[:-1]+data['matches']
        else:
            matches = data['matches']
        last_match = matches[-1]['match_id']
    return matches

In [11]:
my_account_id = '141393206'
print(len(get_match_history(my_account_id, api_key)['matches']))
print(len(get_all_match_history(my_account_id, api_key)))

25
500


## Analyze a match: Who is MVP?

In this part, we will design an evaluating system to find who is the Most Valuable Player. First of all, I want to briefly introduce some basic mechanics of Dota 2. Most of information below are cited from [Dota 2 Wiki](https://dota2.gamepedia.com/).
### Radiant and Dire

In a Dota 2 match, 10 players are divided into two faction: **Radiant** and **Dire**. Each faction occupies a diagonal half of the map, separated in the middle by the river. Two factions are fighting against each other to defense their ancients. (DotA = Defense of the Ancients). 
![](https://d1u5p3l4wpay3k.cloudfront.net/dota2_gamepedia/thumb/1/18/Game_map_7.00.jpg/300px-Game_map_7.00.jpg?version=fe24062355bbafb271a204ab8f369c26?raw=true)

In the players list, the first five belong to Radiant while the last five belong to Dire.
### Gold

**Gold** is the currency used to buy items or instantly revive your hero. Gold can be earned from killing heroes, creeps, or buildings. By adding `gold` and `gold_spent`, we could get the amount of player's total gold at the end of the match.

### Last-hitting and Denying
- **Last-hitting** is a technique where you (or a creep under your control) get the 'last hit' on a neutral creep, enemy lane creep, or enemy hero. The hero that dealt the killing blow to the enemy unit will be awarded a bounty.
- **Denying** is the act of preventing enemy heroes from getting the last hit on a friendly unit by last hitting the unit oneself.

### Kills, Deaths and Assists
- **Kills** is the amount of last-hits to enemies' heroes
- **Deaths** is the amount of last-hits to you by enemies' heroes
- **Assist** is counted when you allied heroes within 1300 radius to kill a enemy.

We usually use *KDA* ratio to evaluate the general performance of a player. It is calculated as
    
    KDA = (# of Kills + # of Assists) / (# of Deaths + 1)

### Damage
**Damage** is any means by which unit's current health can be reduced.
- **Hero Damage** is the amount of damage this player had dealt to enemies' heroes
- **Tower Damage** is the amount of damage this player had dealt to enemies' buildings including towers, shrines and etc.

### Healing

Increasing a unit's current health is most commonly known as **Healing** a unit. 
- **Hero Healing** is the amount of healing this player had made to alies' heroes

------
Now, we could design more criterias that measures players' contribution in a Dota 2 match.
### In Battle Ratio
In battle ratio could effectively show how each player is involved in the team kills:

    In Battle Ratio = (# of Kills + # of Assists) / (# of Team Kills) * 100%

### Damage Ratio
Damage ratio could measure each player's contribution to the total damage dealt to enemies:
    
    Damage Ratio = (# of Hero Damage) / (# of Total Hero Damage Dealt to Enemies) * 100%

### Gold Ratio
Gold ratio could reveal how the team resources are allocated to players:
    
    Gold Ratio = (# of Gold Accquired) / (# of Toal Gold of Team) * 100%
### Gold to Damage Ability
Gold to damage ability is calculated as

    Gold to Damage Ability = Damage Ratio / Gold Ratio
    
This value measures the ability to convert gold into damages of each player (also related to heroes, but we will not consider this factor here). Here, we could directly use this value to find the Most Valuable Player.

---

The `process_match` function below will read in the data of a Dota 2 match and calculate all criteria mentioned above for each player.

In [12]:
def process_match(data):
    cnt = 0
    total_hero_damage = [0, 0]
    total_kill = [0, 0]
    total_gold = [0, 0]
    total_tower_damage = [0, 0]
    for player in data['players']:
        cnt += 1
        player['overall_gold'] = player['gold']+player['gold_spent']
        if (cnt <= 5):
            player['camp'] = 0
            total_hero_damage[0] += player['hero_damage']
            total_kill[0] += player['kills']
            total_gold[0] += player['overall_gold']
        else:
            player['camp'] = 1
            total_hero_damage[1] += player['hero_damage']
            total_kill[1] += player['kills']
            total_gold[1] += player['overall_gold']
    for player in data['players']:
        camp = player['camp']
        player['KDA'] = (player['kills']+player['assists'])/(player['deaths']+1.0)
        if (total_hero_damage[camp] == 0):
            player['hero_damage_per'] = 0.0
        else:
            player['hero_damage_per'] = player['hero_damage']/(total_hero_damage[camp]*1.0)*100.0
        if (total_kill[camp] == 0):
            player['in_battle_per'] = 0.0
        else:
            player['in_battle_per'] = (player['kills']+player['assists'])/(total_kill[camp])*100.0
        player['gold_per'] = player['overall_gold']/(total_gold[camp]*1.0)*100.0
    return data

In [13]:
processed_match = process_match(get_match_details(3777836789, api_key))
print(len(processed_match['players'][0]))

36


### Visualization
We will use Markdown table to visualize a match. 

- `printmd` function would display a markdown script
- `get_png_md` would generate the markdown script to display a URL png
- `show_match` would generate the markdown script to visualize a Dota 2 match

`printmd` is only effective when you run the function. In static files, the displayed result would become `<IPython.core.display.Markdown object>`. Therefore, I've copied my running result (Markdown scripts) into the markdown cells in Appendix which is generated by `show_match` function. Same reason for `show_fav_list`.

In [14]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
def get_png_md(png_url):
    return '![](%s?raw=true)'%png_url

In [15]:
def show_match(data):
    cols = 'Players|Heroes|KDA|Last Hits/Denies|Tower Damage|Hero Healing|In Battle %|Damage %|Gold %|MVP|\n'\
           ':-|:-|:-|:-|:-|:-|:-|:-|:-|:-|\n'
    fmat = '%s|%s**lvl %d**|**%.1f**(%d/%d/%d)|%d/%d|%d|%d|%.1f%%|%.1f%%|%.1f%%|%.1f|\n'
    if (data['radiant_win']):
        radiant_res = ' (Victory)'
        dire_res = ''
    else:
        radiant_res = ''
        dire_res = ' (Victory)'
    md = '## Match Detail\n\n'
    md += '- MatchID: %s\n'%data['match_id']
    md += '- Start Time: %s\n'%\
            (datetime.datetime.fromtimestamp(int(data['start_time'])).strftime('%Y-%m-%d %H:%M:%S'))
    md += '- Duration: %d min\n'%(int(data['duration'])//60)
    md += '***\n'
    md += '### Radiant%s\n\n'%radiant_res
    md += cols
    cnt = 0
    for player in data['players']:
        md += fmat%(player['account_id'], \
                    get_png_md(heroes[heroid2idx[player['hero_id']]]['portrait_url']), \
                    player['level'], \
                    player['KDA'], \
                    player['kills'], \
                    player['deaths'], \
                    player['assists'], \
                    player['last_hits'], \
                    player['denies'], \
                    player['tower_damage'], \
                    player['hero_healing'], \
                    player['in_battle_per'], \
                    player['hero_damage_per'], \
                    player['gold_per'], \
                    player['hero_damage_per']/player['gold_per'])
        cnt += 1
        if (cnt == 5):
            md += '***\n### Dire%s\n\n'%dire_res
            md += cols
    return md
printmd(show_match(processed_match))

## Match Detail

- MatchID: 3777836789
- Start Time: 2018-03-12 15:21:34
- Duration: 54 min
***
### Radiant

Players|Heroes|KDA|Last Hits/Denies|Tower Damage|Hero Healing|In Battle %|Damage %|Gold %|MVP|
:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|
136383633|![](http://cdn.dota2.com/apps/dota2/images/heroes/phantom_assassin_sb.png?raw=true)**lvl 25**|**5.0**(29/8/16)|520/18|4803|0|83.3%|37.6%|38.1%|1.0|
141393206|![](http://cdn.dota2.com/apps/dota2/images/heroes/bristleback_sb.png?raw=true)**lvl 25**|**1.9**(11/13/16)|268/12|396|0|50.0%|18.3%|21.7%|0.8|
142099634|![](http://cdn.dota2.com/apps/dota2/images/heroes/vengefulspirit_sb.png?raw=true)**lvl 24**|**2.3**(2/11/26)|53/7|188|0|51.9%|8.7%|12.0%|0.7|
134202447|![](http://cdn.dota2.com/apps/dota2/images/heroes/zuus_sb.png?raw=true)**lvl 25**|**3.4**(9/10/28)|174/4|250|80|68.5%|30.5%|13.4%|2.3|
136312013|![](http://cdn.dota2.com/apps/dota2/images/heroes/legion_commander_sb.png?raw=true)**lvl 23**|**0.9**(3/13/10)|249/9|569|924|24.1%|4.9%|14.8%|0.3|
***
### Dire (Victory)

Players|Heroes|KDA|Last Hits/Denies|Tower Damage|Hero Healing|In Battle %|Damage %|Gold %|MVP|
:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|
58519674|![](http://cdn.dota2.com/apps/dota2/images/heroes/juggernaut_sb.png?raw=true)**lvl 25**|**5.2**(24/7/18)|374/12|8069|22728|77.8%|37.7%|32.8%|1.1|
160999411|![](http://cdn.dota2.com/apps/dota2/images/heroes/dark_willow_sb.png?raw=true)**lvl 25**|**3.2**(11/12/30)|49/2|1319|0|75.9%|17.0%|16.5%|1.0|
119448368|![](http://cdn.dota2.com/apps/dota2/images/heroes/pudge_sb.png?raw=true)**lvl 22**|**2.5**(4/10/24)|23/0|838|1682|51.9%|10.7%|12.8%|0.8|
89702175|![](http://cdn.dota2.com/apps/dota2/images/heroes/undying_sb.png?raw=true)**lvl 25**|**1.9**(5/18/31)|96/1|808|1282|66.7%|12.1%|12.4%|1.0|
205436003|![](http://cdn.dota2.com/apps/dota2/images/heroes/obsidian_destroyer_sb.png?raw=true)**lvl 25**|**4.4**(10/7/25)|343/18|3438|0|64.8%|22.5%|25.6%|0.9|


## Analyze a player: Favorite Heroes

### Data Collection and processing
In this part, we will focus on analyzing a player from his recent matches. First of all, we need to process each recent match individually and collect all processed data. `get_all_match_history_detail` will read in a player's match history and then return the list of all match details.

It takesapproximated 3 minutes to process 500 matches. 

In [16]:
def get_all_match_history_detail(hist, api_key):
    hist_details = []
    for i in tqdm(range(len(hist))):
        match = hist[i]
        hist_details += [process_match(get_match_details(match['match_id'], api_key))]
    return hist_details

In [17]:
my_hist = get_all_match_history(my_account_id, api_key, game_mode = 1)

In [18]:
my_hist_detail = get_all_match_history_detail(my_hist, api_key)

100%|██████████| 500/500 [03:58<00:00,  2.10it/s]


Using the detailed match history, we want to find a player's most favorite heroes and their average performance. In function `fav_heroes`, we will calculate the following data:

- How many times this player had played this hero
- Winning rate
- Average Kills/Deaths/Assists
- Average KDA
- The average amount of gold earned per minute
- The average amount of experience earned per minute
- Average MVP value (Gold to Damage Ability)

In [19]:
def fav_heroes(hist_detail, account_id):
    counter = {}
    for hero in heroes:
        #cnt, win, kill, death, assist, kda
        counter[hero['id']] = {'cnt' : 0, \
                               'win' : 0, \
                               'kills' : 0, \
                               'deaths' : 0, \
                               'assists' : 0, \
                               'KDA' : 0.0, \
                               'GPM' : 0, \
                               'XPM' : 0, \
                               'MVP' : 0.0}
    for match_detail in hist_detail:
        for player in match_detail['players']:
            if (str(player['account_id']) == str(account_id)):
                counter[player['hero_id']]['cnt'] += 1
                counter[player['hero_id']]['win'] += player['camp'] ^ match_detail['radiant_win']
                counter[player['hero_id']]['kills'] += player['kills']
                counter[player['hero_id']]['deaths'] += player['deaths']
                counter[player['hero_id']]['assists'] += player['assists']
                counter[player['hero_id']]['KDA'] += player['KDA']
                counter[player['hero_id']]['GPM'] += player['gold_per_min']
                counter[player['hero_id']]['XPM'] += player['xp_per_min']
                counter[player['hero_id']]['MVP'] += player['hero_damage_per']*1.0/player['gold_per']
    return counter
counter = fav_heroes(my_hist_detail, my_account_id)
flag = 0
for hero in heroes:
    if (counter[hero['id']]['cnt'] != 0):
        flag += 1
        print("%s:%d"%(hero['localized_name'], counter[hero['id']]['cnt']))
        if (flag > 3): break

Anti-Mage:4
Axe:4
Bloodseeker:8
Crystal Maiden:3


### Visualization
similarly, we will use Markdown table to visualize the most favorite heroes list. `show_fav_list` would generate the markdown script to visualize the list.

In [20]:
def show_fav_list(counter, topN = 10):
    fav_list = sorted(counter.items(), key=lambda x:-x[1]['cnt'])
    cols = 'Heroes|Match Played|Win Rate|KDA(K/D/A)|Avg. KDA|Avg. GPM|Avg. XPM|Avg. MVP|\n'\
           ':-|:-|:-|:-|:-|:-|:-|\n'
    fmat = '%s|%d|%.1f%%|**%.1f**(%.1f/%.1f/%.1f)|%.1f|%.1f|%.1f|%.1f|\n'
    if (topN > len(counter)):
        topN = len(counter)
    md = ''
    md += '### Most Played Heroes\n\n'
    md += cols
    for idx in range(topN):
        md += fmat%(get_png_md(heroes[heroid2idx[fav_list[idx][0]]]['portrait_url']), \
                    fav_list[idx][1]['cnt'], \
                    fav_list[idx][1]['win']/(fav_list[idx][1]['cnt']*1.0)*100.0, \
                    (fav_list[idx][1]['kills']+fav_list[idx][1]['assists'])/(fav_list[idx][1]['deaths']+1.0), \
                    fav_list[idx][1]['kills']/(fav_list[idx][1]['cnt']*1.0), \
                    fav_list[idx][1]['deaths']/(fav_list[idx][1]['cnt']*1.0), \
                    fav_list[idx][1]['assists']/(fav_list[idx][1]['cnt']*1.0), \
                    fav_list[idx][1]['KDA']/(fav_list[idx][1]['cnt']*1.0), \
                    fav_list[idx][1]['GPM']/(fav_list[idx][1]['cnt']*1.0), \
                    fav_list[idx][1]['XPM']/(fav_list[idx][1]['cnt']*1.0), \
                    fav_list[idx][1]['MVP']/(fav_list[idx][1]['cnt']*1.0))
    return md
printmd(show_fav_list(counter))

### Most Played Heroes

Heroes|Match Played|Win Rate|KDA(K/D/A)|Avg. KDA|Avg. GPM|Avg. XPM|Avg. MVP|
:-|:-|:-|:-|:-|:-|:-|
![](http://cdn.dota2.com/apps/dota2/images/heroes/juggernaut_sb.png?raw=true)|34|64.7%|**5.2**(10.6/4.5/12.6)|5.7|579.0|596.6|0.9|
![](http://cdn.dota2.com/apps/dota2/images/heroes/mirana_sb.png?raw=true)|26|65.4%|**5.0**(8.9/4.7/14.8)|5.8|485.2|505.8|0.8|
![](http://cdn.dota2.com/apps/dota2/images/heroes/pudge_sb.png?raw=true)|24|66.7%|**3.1**(6.3/7.4/16.5)|3.1|391.2|472.1|0.9|
![](http://cdn.dota2.com/apps/dota2/images/heroes/nevermore_sb.png?raw=true)|21|81.0%|**4.8**(9.7/4.1/10.5)|7.0|595.9|600.0|1.0|
![](http://cdn.dota2.com/apps/dota2/images/heroes/rubick_sb.png?raw=true)|18|38.9%|**2.4**(2.9/6.6/12.9)|3.1|326.4|331.8|1.0|
![](http://cdn.dota2.com/apps/dota2/images/heroes/luna_sb.png?raw=true)|17|58.8%|**3.7**(7.3/5.5/13.4)|4.0|589.1|568.4|0.9|
![](http://cdn.dota2.com/apps/dota2/images/heroes/legion_commander_sb.png?raw=true)|16|68.8%|**3.4**(7.6/5.4/10.8)|3.5|458.2|499.0|0.7|
![](http://cdn.dota2.com/apps/dota2/images/heroes/ursa_sb.png?raw=true)|15|46.7%|**3.9**(13.4/6.9/13.8)|5.4|513.0|551.2|1.0|
![](http://cdn.dota2.com/apps/dota2/images/heroes/life_stealer_sb.png?raw=true)|14|64.3%|**4.9**(10.9/4.8/13.1)|5.1|549.4|572.6|0.9|
![](http://cdn.dota2.com/apps/dota2/images/heroes/slark_sb.png?raw=true)|14|57.1%|**4.3**(9.4/4.6/10.9)|4.5|502.0|542.4|0.9|


## Summary and references

This tutorial introduced the Valve Dota 2 API and some methods to analyze these data. With basic calculation and visualization, we could make the data much more readable to users. 

Also, there are many designs in this tutorial could be improved (remove the factor of hero differences from player evaluation, etc.). Much more detail about Dota 2 APIs and data analyzing from following links.

1.Dota 2 API: https://wiki.teamfortress.com/wiki/WebAPI#Dota_2

2.Dotamax: http://www.dotamax.com/home/

3.Dota Plus: https://www.dota2.com/plus

# Appendix: `prindmd` result

## Match Detail

- MatchID: 3777836789
- Start Time: 2018-03-12 15:21:34
- Duration: 54 min
***
### Radiant

Players|Heroes|KDA|Last Hits/Denies|Tower Damage|Hero Healing|In Battle %|Damage %|Gold %|MVP|
:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|
136383633|![](http://cdn.dota2.com/apps/dota2/images/heroes/phantom_assassin_sb.png?raw=true)**lvl 25**|**5.0**(29/8/16)|520/18|4803|0|83.3%|37.6%|38.1%|1.0|
141393206|![](http://cdn.dota2.com/apps/dota2/images/heroes/bristleback_sb.png?raw=true)**lvl 25**|**1.9**(11/13/16)|268/12|396|0|50.0%|18.3%|21.7%|0.8|
142099634|![](http://cdn.dota2.com/apps/dota2/images/heroes/vengefulspirit_sb.png?raw=true)**lvl 24**|**2.3**(2/11/26)|53/7|188|0|51.9%|8.7%|12.0%|0.7|
134202447|![](http://cdn.dota2.com/apps/dota2/images/heroes/zuus_sb.png?raw=true)**lvl 25**|**3.4**(9/10/28)|174/4|250|80|68.5%|30.5%|13.4%|2.3|
136312013|![](http://cdn.dota2.com/apps/dota2/images/heroes/legion_commander_sb.png?raw=true)**lvl 23**|**0.9**(3/13/10)|249/9|569|924|24.1%|4.9%|14.8%|0.3|
***
### Dire (Victory)

Players|Heroes|KDA|Last Hits/Denies|Tower Damage|Hero Healing|In Battle %|Damage %|Gold %|MVP|
:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|
58519674|![](http://cdn.dota2.com/apps/dota2/images/heroes/juggernaut_sb.png?raw=true)**lvl 25**|**5.2**(24/7/18)|374/12|8069|22728|77.8%|37.7%|32.8%|1.1|
160999411|![](http://cdn.dota2.com/apps/dota2/images/heroes/dark_willow_sb.png?raw=true)**lvl 25**|**3.2**(11/12/30)|49/2|1319|0|75.9%|17.0%|16.5%|1.0|
119448368|![](http://cdn.dota2.com/apps/dota2/images/heroes/pudge_sb.png?raw=true)**lvl 22**|**2.5**(4/10/24)|23/0|838|1682|51.9%|10.7%|12.8%|0.8|
89702175|![](http://cdn.dota2.com/apps/dota2/images/heroes/undying_sb.png?raw=true)**lvl 25**|**1.9**(5/18/31)|96/1|808|1282|66.7%|12.1%|12.4%|1.0|
205436003|![](http://cdn.dota2.com/apps/dota2/images/heroes/obsidian_destroyer_sb.png?raw=true)**lvl 25**|**4.4**(10/7/25)|343/18|3438|0|64.8%|22.5%|25.6%|0.9|

### Most Played Heroes

Heroes|Match Played|Win Rate|KDA(K/D/A)|Avg. KDA|Avg. GPM|Avg. XPM|
:-|:-|:-|:-|:-|:-|:-|
![](http://cdn.dota2.com/apps/dota2/images/heroes/juggernaut_sb.png?raw=true)|34|64.7%|**5.2**(10.6/4.5/12.6)|5.7|579.0|596.6|
![](http://cdn.dota2.com/apps/dota2/images/heroes/mirana_sb.png?raw=true)|26|65.4%|**5.0**(8.9/4.7/14.8)|5.8|485.2|505.8|
![](http://cdn.dota2.com/apps/dota2/images/heroes/pudge_sb.png?raw=true)|22|68.2%|**3.0**(6.0/7.5/16.4)|2.9|395.7|476.2|
![](http://cdn.dota2.com/apps/dota2/images/heroes/nevermore_sb.png?raw=true)|21|81.0%|**4.8**(9.7/4.1/10.5)|7.0|595.9|600.0|
![](http://cdn.dota2.com/apps/dota2/images/heroes/rubick_sb.png?raw=true)|18|38.9%|**2.4**(2.9/6.6/12.9)|3.1|326.4|331.8|
![](http://cdn.dota2.com/apps/dota2/images/heroes/luna_sb.png?raw=true)|17|58.8%|**3.7**(7.3/5.5/13.4)|4.0|589.1|568.4|
![](http://cdn.dota2.com/apps/dota2/images/heroes/legion_commander_sb.png?raw=true)|16|68.8%|**3.4**(7.6/5.4/10.8)|3.5|458.2|499.0|
![](http://cdn.dota2.com/apps/dota2/images/heroes/life_stealer_sb.png?raw=true)|14|64.3%|**4.9**(10.9/4.8/13.1)|5.1|549.4|572.6|
![](http://cdn.dota2.com/apps/dota2/images/heroes/ursa_sb.png?raw=true)|14|50.0%|**3.9**(13.6/6.9/13.7)|5.6|518.0|554.2|
![](http://cdn.dota2.com/apps/dota2/images/heroes/slark_sb.png?raw=true)|14|57.1%|**4.3**(9.4/4.6/10.9)|4.5|502.0|542.4|

<IPython.core.display.Markdown object>